## Review dashboard

https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/US48/US48

## Get desired data endpoint from API

https://www.eia.gov/opendata/browser/electricity/rto/fuel-type-data

## Initialize route in Python

In [54]:
from eia import EIAClient

client = EIAClient() 
route = client.get_data_endpoint('electricity/rto/fuel-type-data')

2025-12-10 15:36:14,832 - INFO - EIAClient initialized.
2025-12-10 15:36:14,834 - INFO - Directly accessing data endpoint metadata for: electricity/rto/fuel-type-data
2025-12-10 15:36:14,834 - INFO - Fetching metadata for route: electricity/rto/fuel-type-data


https://www.eia.gov/opendata/browser/electricity/rto/region-sub-ba-data?frequency=hourly&data=value;&sortColumn=period;&sortDirection=desc;

## Filtering options

In [55]:
route.get?

Signature:
route.get(
    data_columns: Optional[List[str]] = None,
    facets: Optional[Dict[str, Union[str, List[str]]]] = None,
    frequency: Optional[str] = None,
    start: Optional[str] = None,
    end: Optional[str] = None,
    sort: Optional[List[Dict[str, str]]] = None,
    length: Optional[int] = None,
    offset: Optional[int] = None,
    output_format: Optional[Literal['json', 'xml']] = 'json',
    paginate: bool = True,
) -> pandas.core.frame.DataFrame
Docstring:
Retrieves data from this endpoint, stores it and metadata internally,
and returns the data as a pandas DataFrame. Handles pagination automatically by default.

Args:
    data_columns: List of data column IDs to retrieve. If None, all available columns are fetched.
    facets: Dictionary of facet filters, e.g., {'stateid': 'CA', 'sectorid': ['RES', 'COM']}
    frequency: Data frequency ID (e.g., 'daily', 'monthly')
    start: Start date/period
    end: End date/period
    sort: List of sort specifications
    leng

### Facets: column values

In [56]:
route.facets

FacetContainer(facets=[respondent, fueltype])

In [57]:
facet_options = {}

for name, facet in route.facets.items():
    values = facet.get_values()
    options = []
    for meta in values:
        options.append(meta.__dict__)
    
    facet_options[name] = options

2025-12-10 15:36:16,006 - INFO - Fetching facet values for facet 'respondent' in route: electricity/rto/fuel-type-data
2025-12-10 15:36:16,546 - INFO - Fetching facet values for facet 'fueltype' in route: electricity/rto/fuel-type-data


In [58]:
import yaml

with open("facet_options.yml", "w") as f:
    yaml.dump(facet_options, f, sort_keys=False)

### Frequencies: datetime intervals

In [59]:
pd.DataFrame(route.frequencies)

,id,description,query,format
0,hourly,One data point for each hour in UTC time.,H,"YYYY-MM-DD""T""HH24"
1,local-hourly,One data point for each hour in local time.,LH,"YYYY-MM-DD""T""HH24TZH"


## Query data

In [60]:
df = route.get(
    data_columns=['value'],
    facets={'respondent': 'MISO'},
    frequency='local-hourly',
    start='2024-01-01T00-05',
    end='2025-01-01T00-05',
    sort=[
        {'column': 'period','direction': 'desc'},
        {'column': 'fueltype','direction': 'asc'}
    ]
)


2025-12-10 15:36:17,163 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 0, Length: All (Pagination)
2025-12-10 15:36:17,164 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-12-10 15:36:20,906 - INFO - Received 5000 rows.
2025-12-10 15:36:20,907 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 5000, Length: All (Pagination)
2025-12-10 15:36:20,907 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-12-10 15:36:24,214 - INFO - Received 5000 rows.
2025-12-10 15:36:24,215 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 10000, Length: All (Pagination)
2025-12-10 15:36:24,215 - INFO - Fetching data for route: electricity/rto/fuel-type-data/data
2025-12-10 15:36:55,752 - INFO - Received 5000 rows.
2025-12-10 15:36:55,755 - INFO - Fetching data for route: electricity/rto/fuel-type-data - Offset: 15000, Length: All (Pagination)
2025-12-10 15:36:55,755 - INFO - Fetchin

## Preprocess data

In [61]:
import pandas as pd

In [62]:
x = pd.to_datetime(df.period, utc=True)
x

0       2025-01-01 05:00:00+00:00
1       2025-01-01 05:00:00+00:00
                   ...           
61490   2024-01-01 05:00:00+00:00
61491   2024-01-01 05:00:00+00:00
Name: period, Length: 61492, dtype: datetime64[ns, UTC]

In [63]:
x = x.dt.tz_convert('America/New_York')
x

0       2025-01-01 00:00:00-05:00
1       2025-01-01 00:00:00-05:00
                   ...           
61490   2024-01-01 00:00:00-05:00
61491   2024-01-01 00:00:00-05:00
Name: period, Length: 61492, dtype: datetime64[ns, America/New_York]

In [64]:
df.period = x
df

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2025-01-01 00:00:00-05:00,MISO,"Midcontinent Independent System Operator, Inc.",COL,Coal,17696.0,megawatthours
1,2025-01-01 00:00:00-05:00,MISO,"Midcontinent Independent System Operator, Inc.",NG,Natural Gas,17929.0,megawatthours
...,...,...,...,...,...,...,...
61490,2024-01-01 00:00:00-05:00,MISO,"Midcontinent Independent System Operator, Inc.",WAT,Hydro,777.0,megawatthours
61491,2024-01-01 00:00:00-05:00,MISO,"Midcontinent Independent System Operator, Inc.",WND,Wind,6872.0,megawatthours


## Export data

In [66]:
df.to_parquet('../fuel_type_data_detroit.parquet')